# Simulate the trading with historical data

## Import modules and define functions

In [1]:
from collections import defaultdict
import robin_stocks as r
import pickle
import pandas as pd
import yfinance as yf
from datetime import date
import logging

pd.set_option('display.max_rows',100)
pd.set_option('display.max_columns',20)
pd.set_option('display.max_colwidth',100)
pd.set_option('display.width',None)

def model_1(x,buy_or_sell):
    if buy_or_sell=='buy':
        ## buy model 1: price 100 factor 1; price 20 factor 2
        return round(-1/100*x+2,2) if x<=100 else 1
    elif buy_or_sell=='sell':
        #=IF(H5<100/0.9, -0.006*H5+1.5,0.9)
        return round(-0.006*x+1.5,2) if x/0.9<=100 else 0.9
    else:
        print('wrong parameter')
        
def model_2(x,buy_or_sell):
    if buy_or_sell=='buy':
        ## buy model 1: price 100 factor 1; price 20 factor 2
        return 1
    elif buy_or_sell=='sell':
        #=IF(H5<100/0.9, -0.006*H5+1.5,0.9)
        return 0.9
    else:
        print('wrong parameter')

def two_points_line(p1,p2):
    # y= a*x+b
    a=(p2[1]-p1[1])/(p2[0]-p1[0])
    b=p1[1]-(p2[1]-p1[1])/(p2[0]-p1[0])*p1[0]
    print('a is {0}, b is {1} , y = {0}*x + {1} '.format(a,b))

def convert_df_to_list(df): # df has columns [Open, High, Low, Close, Adj Close,Volume], index is Date
    data_list=[]
    for i in range(len(df)):
        tem=df.iloc[i].tolist()[0:4]
        data_list.append(tem[0]) #open
        data_list.append(tem[1]) #high
        data_list.append(tem[2]) #low
        data_list.append(tem[3]) #close
    return data_list

# verify profit
def verify_profit(year):
    d=stockTrades[year].tradingRecords
    total_buy=0
    total_sell=0
    for key in d:
        l=d[key]
        for r in l:
            if r[0]=='buy':
                total_buy+=r[1]*key
            else:
                total_sell+=r[1]*key
    print(total_buy)
    print(total_sell)
    print('profit is, ',stockTrades[year].openShares*stockTrades[year].currentPrice+total_sell-total_buy)

## Create logger 

In [32]:
logger=logging.getLogger('simulation_trading')
logger.setLevel(logging.DEBUG)
formatter=logging.Formatter('%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s')

# file handler save logs to file
file_handler=logging.FileHandler(filename='simulation_trading.log',mode='w')
file_handler.setLevel(logging.INFO)
file_handler.setFormatter(formatter)

# stream handler print logs on console
stream_handler=logging.StreamHandler()
stream_handler.setFormatter(formatter)

logger.addHandler(file_handler)
logger.addHandler(stream_handler)


# Log in

In [3]:
#log and lout
import getpass
account=getpass.getpass('Enter your account')
psd = getpass.getpass('Enter your password')
log=r.authentication.login('hejiakoo@gmail.com',psd)
#r.logout()

Enter your account ··················
Enter your password ··············


## Define the simulation class

In [4]:
class StockTrade:
    def __init__(self,symbol,baseAmount,discountFactor,currentPrice,initialShares,model,realtrade=False):
        """
        readltrade(bool): if realtrade ==True, will send orders to robinhood;otherwise, it's a simulation
        """
        self.realtrade=realtrade
        self.symbol=symbol
        self.baseAmount=baseAmount
        self.discountFactor=discountFactor
        self.priceRange=self.generate_price_range()
        self.buy_factor_list=self.generate_factor_list(model,'buy') # buy model
        self.sell_factor_list=self.generate_factor_list(model,'sell') # sell model
        self.buy_amount_list=self.generate_buy_amount_list()
        self.sell_amount_list=self.generate_sell_amount_list()
        self.combined_lists=self.initialize_combined_list(currentPrice)
        self.openOrders={}
        self.openShares=initialShares
        self.orderNumber=0
        self.totalMoneyBuy=self.openShares*currentPrice
        self.totalmoneySell=0
        self.max_money_needed=self.totalMoneyBuy
        self.max_loss=0
        self.currentPrice=currentPrice
        #self.profit=0
        self.tradingRecords=defaultdict(list)
        self.update_tradingRecords(currentPrice,'buy',initialShares)# update the intial buy to account
        print('initialize, bought {0} shares at {1} worth {2}'.format(initialShares,currentPrice,initialShares*currentPrice))
        self.set_order()

                
    def generate_price_range(self): # return a list
        base=100
        less=100
        more=100
        l=[100]
        while more <200:
            more=more/(1-self.discountFactor)
            l.append(round(more,2))
        while less >20:
            less=less*(1-self.discountFactor)
            l.append(round(less,2))
        return sorted(l,reverse=True)
    
    def generate_factor_list(self,model,buy_or_sell):
        return [model(x,buy_or_sell) for x in self.priceRange]        
        
    def generate_buy_amount_list(self):
        return [round(self.baseAmount*i,2) for i in self.buy_factor_list]
    def generate_sell_amount_list(self):
        return [round(self.baseAmount*i,2) for i in self.sell_factor_list]
#     def initialize_labels(self,currentPrice):
#         l=[]
#         self.labels=['buy' if i>currentPrice-10 else 'sell' for i in self.priceRange]
    def get_profit(self,currentPrice):
        return round(self.openShares*currentPrice+self.totalmoneySell-self.totalMoneyBuy,2)
    
    def get_latest_price(self): # get the latest price of this symbel
        return float(r.stocks.get_latest_price(self.symbol)[0])
    
    def initialize_combined_list(self, currentPrice):
        """
        [{'price':* ,'sell':*, 'buy':*}]  item in list,
        sell: mean if it's allowed to sell at this price. affected by lower-price position buy order
        buy: mean if it's allowed to buy at his price. Affected by higher-price position sell order
        """
        l=[] 
        for i in range(len(self.priceRange)):
            price=self.priceRange[i]
            if self.priceRange[i]>=currentPrice:
                d={'price':price,'buy':False,'buy_amount':self.buy_amount_list[i],'sell':True,'sell_amount':self.sell_amount_list[i]}
            else:
                d={'price':price,'buy':True,'buy_amount':self.buy_amount_list[i],'sell':False,'sell_amount':self.sell_amount_list[i]}
            l.append(d)
#         for price in self.priceRange:
#             if price >=currentPrice:
#                 d={'price':price,'sell':True, 'buy':False}
#             else:
#                 d={'price':price,'sell':False, 'buy':True}
#             l.append(d)
        return l       
                
    def sell_setting(self,position):

        logger.info('sell setting: ')
        price=self.priceRange[position]
        amount=self.sell_amount_list[position]
        shares=round(amount/price)
        self.check_shares_avai_for_sale(shares)
        logger.info('sell {0} shares, amount {1} at {2}'.format(shares,amount,price))
        #r.orders.order_sell_limit('TQQQ',quantity=shares,limitPrice=price,extendedHours=True)
        self.openOrders.update({'sell':[{'shares':shares,'amount':amount,'price':price,'position':position}]})
        
    def buy_setting(self,position):
        logger.info('buy setting: ')
        price=self.priceRange[position]
        amount=self.buy_amount_list[position]
        shares=round(amount/price)
        logger.info('buy {0} shares, amount {1} at {2}'.format(shares,amount,price))
        #r.orders.order_buy_limit('TQQQ',quantity=shares,limitPrice=price,extendedHours=True)
        # openOrders is a dictionary, whose keys include 'buy' and 'sell'
        self.openOrders.update({'buy':[{'shares':shares,'amount':amount,'price':price,'position':position}]})
  
    def set_order(self):
        logger.info('---------set up open orders-----------')
        for j in range(len(self.priceRange)):
            if self.combined_lists[j]['buy']==True:
                self.buy_setting(j)
                break
        for k in range(len(self.priceRange)-1,-1,-1):
            if self.combined_lists[k]['sell']==True:
                self.sell_setting(k)
                break
        if self.realtrade==True:
            self.send_orders_to_rb() # send orders to robinhood
        
    def send_orders_to_rb(self):
        logger.warning('send orders to robinhood')
        r.orders.cancel_all_stock_orders() # clean all orders
        open_orders=self.openOrders
        limit_put_args={'symbol':'TQQQ','quantity':open_orders['buy'][0]['shares'],'limitPrice':open_orders['buy'][0]['price'],'extendedHours':True}
        r.orders.order_buy_limit(**limit_put_args)  # make the limit buy order 
        limit_sell_args={'symbol':'TQQQ','quantity':open_orders['sell'][0]['shares'],'limitPrice':open_orders['sell'][0]['price'],'extendedHours':True}
        r.orders.order_sell_limit(**limit_sell_args)# make the limit sell order
        logger.warning('---confirm the orders---')
        logger.warning(r.orders.get_all_open_stock_orders()) # check the orders
    def market_buy_rb(self,shares):
        logger.warning('market buy {} shares in robinhood'.format(shares))
        market_buy_args={'symbol':self.symbol,'quantity':shares,'extendedHours':True}
        r.orders.order_buy_market(**market_buy_args)
    
    def update_tradingRecords(self,price,action,shares):
        self.tradingRecords[price].append([action,shares])
    def update_openShares(self,action,shares):
        self.openShares=self.openShares-shares if action=='sell' else self.openShares+shares
    def update_labels(self,position,action,b):
        self.combined_lists[position].update({action:b})
    def check_shares_avai_for_sale (self,shares_to_sell): 
        '''
        when not enough shares available for sale, due to price keeping moving higher, buy some shares one time
        '''
        if self.openShares<=shares_to_sell+1:
            logger.warning('-------------')
            logger.warning('current share is {}, not enough shares in hand available for sale'.format(self.openShares))
            self.increase_shares(int(2*self.baseAmount/self.currentPrice))
            logger.warning('-------------')        
        
    def increase_shares(self,shares):
        logger.warning('buy {0} shares at {1} before open the sell order'.format(shares,self.currentPrice))
        self.update_tradingRecords(self.currentPrice,'buy',shares)
        self.update_openShares('buy',shares)
        self.totalMoneyBuy+=self.currentPrice*shares
        self.update_max_money_needed()
        if self.realtrade==True:
            self.market_buy_rb(shares)
        
    def update_max_money_needed(self):
        self.max_money_needed=max(self.max_money_needed,self.totalMoneyBuy-self.totalmoneySell)
    def update_max_loss_value(self,currentPrice):
        self.max_loss=min(self.get_profit(currentPrice),self.max_loss)                  
    
    def tradeStock(self,currentPrice):
        self.currentPrice=currentPrice
        #print('profit is ,', self.get_profit(currentPrice))
        self.update_max_loss_value(currentPrice)
        if currentPrice >self.openOrders['sell'][0]['price']:
            shares,amount,price,position=self.openOrders['sell'][0].values()
            action='sell'
            logger.info('---------execute order--------')
            #print('current price is ',currentPrice)
            logger.info('before trade, current shares: {}'.format(self.openShares))
            logger.info('now price {0}, sold {1} shares at {2} worth {3}'.format(currentPrice,shares,price,amount))
            #update tradingRecords
            self.totalmoneySell+=price*shares
            self.openOrders.update({'sell':[]})  # erase the open sell order as it is fulfilled
            self.orderNumber+=1
            self.update_tradingRecords(price,action,shares)  
            self.update_openShares(action,shares)
            self.update_labels(position,'sell',False)
            self.update_labels(position+1,'buy',True)
            logger.info('after trade, current shares: {}'.format(self.openShares))
            [logger.info(i) for i in self.combined_lists]
            #print(*self.combined_lists,sep='\n')
            # open new orders
            self.set_order()
            
            
        elif currentPrice<self.openOrders['buy'][0]['price']:  # currentPrice < openOrder buy limit price
            shares,amount,price,position=self.openOrders['buy'][0].values()
            action='buy'
            logger.info('---------execute order--------')
            #print('current price is ',currentPrice)
            logger.info('before trade, current shares: {}'.format(self.openShares))
            logger.info('now price {0}, bought {1} shares at {2} worth {3}'.format(currentPrice,shares,price,amount))
            #update
            self.totalMoneyBuy+=price*shares
            self.update_max_money_needed()
            logger.info('money required {}'.format(self.max_money_needed))
            self.openOrders.update({'buy':[]}) # erase the open buy order as it is fulfilled 
            self.orderNumber+=1
            self.update_tradingRecords(price,action,shares)  
            self.update_openShares(action,shares)          
            self.update_labels(position,'buy',False)
            self.update_labels(position-1,'sell',True)
            logger.info('after trade, current shares: {}'.format(self.openShares))
            [logger.info(i) for i in self.combined_lists]
            # open new orders
            self.set_order() 
        else:
            print('---------no execute --------')

In [5]:
def save_object(obj,filename):
    with open(filename,'wb') as output_data:
        pickle.dump(obj,output_data,pickle.HIGHEST_PROTOCOL)

def load_object(filename):
    with open(filename,'rb') as input_data:
        obj=pickle.load(input_data)
        return obj

In [25]:
df=yf.download('TQQQ',start=date(2020,9,2)) #,end=date(2020,9,10)
current_list=convert_df_to_list(df)

[*********************100%***********************]  1 of 1 completed


In [26]:
df.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-09-09,129.029999,135.199997,125.669998,132.070007,132.070007,36294000
2020-09-10,136.520004,138.389999,121.739998,124.430000,124.430000,58242100
2020-09-11,127.000000,128.190002,116.809998,121.709999,121.709999,48224700
2020-09-14,127.000000,130.809998,125.080002,127.980003,127.980003,33518800
2020-09-15,133.330002,135.130005,132.699997,134.850006,134.850006,5726451


In [33]:
#def __init__(self,symbol,baseAmount,discountFactor,currentPrice,initialShares,model):

stockTrade=StockTrade('TQQQ',2000,0.1,104,30,model=model_1,realtrade=False)
for p in current_list:
    print('---------------------------------------------------------------------')
    print('current price is : {}'.format(p))
    stockTrade.tradeStock(p)

2020-09-15 14:49:00,682,682 simulation_trading INFO ---------set up open orders-----------
2020-09-15 14:49:00,682,682 simulation_trading INFO ---------set up open orders-----------
2020-09-15 14:49:00,684,684 simulation_trading INFO buy setting: 
2020-09-15 14:49:00,684,684 simulation_trading INFO buy setting: 
2020-09-15 14:49:00,685,685 simulation_trading INFO buy 20 shares, amount 2000.0 at 100
2020-09-15 14:49:00,685,685 simulation_trading INFO buy 20 shares, amount 2000.0 at 100
2020-09-15 14:49:00,686,686 simulation_trading INFO sell setting: 
2020-09-15 14:49:00,686,686 simulation_trading INFO sell setting: 
2020-09-15 14:49:00,688,688 simulation_trading INFO sell 16 shares, amount 1800.0 at 111.11
2020-09-15 14:49:00,688,688 simulation_trading INFO sell 16 shares, amount 1800.0 at 111.11
2020-09-15 14:49:00,689,689 simulation_trading INFO ---------execute order--------
2020-09-15 14:49:00,689,689 simulation_trading INFO ---------execute order--------
2020-09-15 14:49:00,690,69

initialize, bought 30 shares at 104 worth 3120
---------------------------------------------------------------------
current price is : 175.47999572753906
---------------------------------------------------------------------
current price is : 175.6699981689453
---------------------------------------------------------------------
current price is : 164.6699981689453
---------------------------------------------------------------------
current price is : 174.52999877929688
---------------------------------------------------------------------
current price is : 166.72999572753906
---------no execute --------
---------------------------------------------------------------------
current price is : 167.39999389648438
---------no execute --------
---------------------------------------------------------------------
current price is : 142.82000732421875
---------no execute --------
---------------------------------------------------------------------
current price is : 147.72999572753906
----

2020-09-15 14:49:00,882,882 simulation_trading INFO after trade, current shares: 35
2020-09-15 14:49:00,883,883 simulation_trading INFO {'price': 209.08, 'buy': False, 'buy_amount': 2000, 'sell': True, 'sell_amount': 1800.0}
2020-09-15 14:49:00,883,883 simulation_trading INFO {'price': 209.08, 'buy': False, 'buy_amount': 2000, 'sell': True, 'sell_amount': 1800.0}
2020-09-15 14:49:00,884,884 simulation_trading INFO {'price': 188.17, 'buy': False, 'buy_amount': 2000, 'sell': True, 'sell_amount': 1800.0}
2020-09-15 14:49:00,884,884 simulation_trading INFO {'price': 188.17, 'buy': False, 'buy_amount': 2000, 'sell': True, 'sell_amount': 1800.0}
2020-09-15 14:49:00,885,885 simulation_trading INFO {'price': 169.35, 'buy': False, 'buy_amount': 2000, 'sell': True, 'sell_amount': 1800.0}
2020-09-15 14:49:00,885,885 simulation_trading INFO {'price': 169.35, 'buy': False, 'buy_amount': 2000, 'sell': True, 'sell_amount': 1800.0}
2020-09-15 14:49:00,886,886 simulation_trading INFO {'price': 152.42, 

---------------------------------------------------------------------
current price is : 141.63999938964844
---------no execute --------
---------------------------------------------------------------------
current price is : 124.94999694824219
---------no execute --------
---------------------------------------------------------------------
current price is : 133.97000122070312
---------no execute --------
---------------------------------------------------------------------
current price is : 121.05999755859375
---------------------------------------------------------------------
current price is : 121.62000274658203
---------no execute --------
---------------------------------------------------------------------
current price is : 129.02999877929688
---------no execute --------
---------------------------------------------------------------------
current price is : 135.1999969482422
---------no execute --------
---------------------------------------------------------------------
c

In [9]:
# check the open orders
stockTrade.openOrders

{'buy': [{'shares': 18, 'amount': 2000, 'price': 111.11, 'position': 6}],
 'sell': [{'shares': 13, 'amount': 1800.0, 'price': 137.17, 'position': 4}]}

In [10]:
# define a function to send orders to robinhood
def send_orders_to_rb():
    r.orders.cancel_all_stock_orders() # clean all orders
    open_orders=stockTrade.openOrders
    limit_put_args={'symbol':'TQQQ','quantity':open_orders['buy'][0]['shares'],'limitPrice':open_orders['buy'][0]['price'],'extendedHours':True}
    r.orders.order_buy_limit(**limit_put_args)  # make the limit buy order 
    limit_sell_args={'symbol':'TQQQ','quantity':open_orders['sell'][0]['shares'],'limitPrice':open_orders['sell'][0]['price'],'extendedHours':True}
    r.orders.order_sell_limit(**limit_sell_args)# make the limit sell order
    print('---confirm the orders---')
    r.orders.get_all_open_stock_orders() # check the orders


In [11]:
#send_orders_to_rb()

## Simulate the trading by testing with different values for parameters

In [13]:
"""
different factors to test: year, discount factor, 
"""
from datetime import date
def get_history_data(years): # years is a list of year, such [2017,2018,2019]
    data_years=dict()
    for year in years:    
        df=yf.download('TQQQ',start=date(year,1,1),end=date(year+1,1,1))
        data_years[year]=df
    return data_years  

#__init__(self,symbol,baseAmount,discountFactor,currentPrice,initialShares)
def test_year_discount_model(year,discount,model):
    df=yf.download('TQQQ',start=date(year,1,1),end=date(year+1,1,1))
    data_list=convert_df_to_list(df)
    start_price,end_price=data_list[0],data_list[-1]
    baseAmount=2000
    discountFactor=discount
    currentPrice=start_price
    initialShares=round(2*2000/currentPrice)
    stockTrade=StockTrade('TQQQ',baseAmount,discountFactor,currentPrice,initialShares,model=model)
    for price in data_list:
        print('current price is ', price)
        stockTrade.currentPrice=price
        stockTrade.tradeStock(price)
    return stockTrade

def check_results(stockTrade):
    print('profit: ',stockTrade.get_profit(stockTrade.currentPrice))
    print('openshare: ', stockTrade.openShares)
    print('openorders: ',stockTrade.openOrders)
    print('total money spent on buy: ',stockTrade.totalMoneyBuy)
    print('total money from sell : ', stockTrade.totalmoneySell)
    print('max money needed', stockTrade.max_money_needed)
    print('max loss', stockTrade.max_loss)
    return 

In [14]:
logger.removeHandler(stream_handler) # don't print on console
import numpy as np
import itertools
years=[2018,2019,2020]
discount_factors=list(range(2,21,4))
models=[model_1,model_2]
iterations=itertools.product(years,discount_factors,models)
stockTrades=dict()
df_years=get_history_data(years)
for it in iterations:
    year,discount,model=it
    name='{}-{}-{}'.format(year,discount,model.__name__)
    stockTrades[name]=test_year_discount_model(year,discount/100,model)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
initialize, bought 85 shares at 46.96666717529297 worth 3992.1667098999023
current price is  46.96666717529297
---------no execute --------
current price is  48.71333312988281
current price is  46.630001068115234
---------no execute --------
current price is  48.61333465576172
current price is  48.810001373291016
---------no execute --------
current price is  50.220001220703125
current price is  48.8033332824707
---------no execute --------
current price is  50.03666687011719
---------no execute --------
current price is  50.62333297729492
current price is  50.8033332824707
---------no execute --------
current price is  50.1533317565918
---------no execute --------
current price is  50.33000183105469
---------no ex

In [17]:
cols=['year','discount','model','profit','totalMoneyBuy','totalmoneySell','max_money_needed','max_loss','openShares','year_start_price','year_end_price']
df=pd.DataFrame(columns=cols)
iterations=itertools.product(years,discount_factors,models)
for it in iterations:
    year,discount,model=it
    name='{}-{}-{}'.format(year,discount,model.__name__)
    print(name)
    stockTrade=stockTrades[name]
    #check_results(stockTrades[name])
    profit=stockTrades[name].get_profit(stockTrades[name].currentPrice)
    totalMoneyBuy=stockTrades[name].totalMoneyBuy
    totalmoneySell=stockTrades[name].totalmoneySell
    max_money_needed=stockTrades[name].max_money_needed
    max_loss=stockTrades[name].max_loss
    openShares=stockTrades[name].openShares
    start_price=df_years[year]['Open'][0]
    end_price=df_years[year]['Close'][-1]
    df.loc[len(df)]=[year,discount,model.__name__,profit,totalMoneyBuy,totalmoneySell,max_money_needed,max_loss,openShares,start_price,end_price]
    #print('year start price: ', start_price)
    #print('year end price: ',end_price)
    

2018-2-model_1
2018-2-model_2
2018-6-model_1
2018-6-model_2
2018-10-model_1
2018-10-model_2
2018-14-model_1
2018-14-model_2
2018-18-model_1
2018-18-model_2
2019-2-model_1
2019-2-model_2
2019-6-model_1
2019-6-model_2
2019-10-model_1
2019-10-model_2
2019-14-model_1
2019-14-model_2
2019-18-model_1
2019-18-model_2
2020-2-model_1
2020-2-model_2
2020-6-model_1
2020-6-model_2
2020-10-model_1
2020-10-model_2
2020-14-model_1
2020-14-model_2
2020-18-model_1
2020-18-model_2


In [18]:
df.head()

,year,discount,model,profit,totalMoneyBuy,totalmoneySell,max_money_needed,max_loss,openShares,year_start_price,year_end_price
0,2018,2,model_1,-43848.45,770791.510330,562966.98,221821.330330,-74623.40,4427,46.966667,37.040001
1,2018,2,model_2,-17082.87,528020.456736,432227.58,106168.746736,-32286.72,2125,46.966667,37.040001
2,2018,6,model_1,-4698.56,185258.050116,128481.25,63755.390116,-15322.88,1406,46.966667,37.040001
3,2018,6,model_2,-1506.06,126435.656780,97371.84,34283.236780,-7366.29,744,46.966667,37.040001
4,2018,10,model_1,-2139.51,85247.650139,51698.22,36129.810139,-8334.83,848,46.966667,37.040001


In [19]:
#df.loc[df['year'] == 2020]
df['profit_over_maxMoney']=df['profit']/df['max_money_needed']
df['maxLoss_over_maxMoney']=df['max_loss']/df['max_money_needed']

In [20]:
df.sort_values(['profit_over_maxMoney'],ascending=False)
#df.sort_values('maxLoss_over_maxMoney',ascending=False)

,year,discount,model,profit,totalMoneyBuy,totalmoneySell,max_money_needed,max_loss,openShares,year_start_price,year_end_price,profit_over_maxMoney,maxLoss_over_maxMoney
19,2019,18,model_2,5607.38,13922.889913,14510.37,4342.939913,0.00,58,34.779999,86.550003,1.291148,0.000000
24,2020,10,model_1,44677.83,90952.319935,74188.71,38730.409935,-13292.34,456,88.709999,134.800095,1.153559,-0.343202
18,2019,18,model_1,7475.68,20718.299650,18240.73,6746.489650,-373.29,115,34.779999,86.550003,1.108084,-0.055331
16,2019,14,model_1,5878.07,20792.179913,23035.15,5785.569913,-524.81,42,34.779999,86.550003,1.015988,-0.090710
28,2020,18,model_1,18166.97,26553.799959,20871.79,18170.839959,-6553.64,177,88.709999,134.800095,0.999787,-0.360668
22,2020,6,model_1,58924.83,176244.320055,146240.75,62575.740055,-25053.85,660,88.709999,134.800095,0.941656,-0.400376
26,2020,14,model_1,25743.39,42306.949959,33422.16,27543.879959,-9859.35,257,88.709999,134.800095,0.934632,-0.357951
25,2020,10,model_2,22375.01,69876.779935,63147.95,26326.069935,-10235.41,216,88.709999,134.800095,0.849918,-0.388794
15,2019,10,model_2,4726.81,24051.869824,26874.58,5659.059824,-176.30,22,34.779999,86.550003,0.835264,-0.031154
29,2020,18,model_2,9805.38,19882.889959,18100.63,12492.449959,-4844.46,86,88.709999,134.800095,0.784904,-0.387791
